In [1]:
#%%
import pandas as pd
import numpy as np
drugs = []
genes = []
parsename = 'updated_dependency_output.txt'                  

df_drug = pd.read_csv('./drugs/drugs.tsv', sep='\t')         
df_genes = pd.read_csv('./genes/genes.tsv', sep='\t')        

drugs.extend(df_drug["Name"])
genes.extend(df_genes["Name"])

#drug names from DrugBank
drugbank_drug= pd.read_csv('./drugs/drugbank vocabulary.csv')   
drugs.extend(drugbank_drug["Common name"])


set_drug = set(drugs)
set_gene = set(genes)
# %%
dependencies = []
infile = open(parsename, 'r')
for line in infile:
    if line[0].islower() and line.find('(') != -1:
        stripline = line[line.find('(') + 1 : line.find(')')]
        stripline = stripline.split(', ')
        for i in range(2):
            stripline[i] = stripline[i][:stripline[i].rfind('-')]
        drug, gene = '', ''
        for word in stripline:
            if word in set_drug:
                drug = word
            elif word in set_gene:
                gene = word
        if drug and gene:
            dependencies.append((line[:line.find('(')], (drug, gene)))

# %%
setrelations = set()
setpairs = set()
for dependency in dependencies:
    setrelations.add(dependency[0])
    setpairs.add(dependency[1])

relations = list(setrelations)
pairs = list(setpairs)

zeroes = np.zeros(shape=(len(pairs), len(relations)))
matrix = pd.DataFrame(zeroes, index = pairs, columns = relations)
# %%
for dependency in dependencies:
    matrix.at[dependency[1], dependency[0]] += 1

df_matrix=pd.DataFrame(matrix)

In [2]:
df_matrix

,amod,conj_or,nn,conj_nor,prep_in,prep_with,conj_plus,appos,prep_from,prep_such_as,...,prep_of,prep_by,prep_under,conj_and,prep_for,prep_on,prep_at,dobj,prep_to,conj_but
"(chymotrypsin, acrosin)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(glucocorticoids, glucagon)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
"(streptokinase, plasminogen)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(thrombin, alpha-2-macroglobulin)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
"(C-peptide, insulin)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(dexamethasone, albumin)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
"(iron, ceruloplasmin)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(ATP, transferrin)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(zinc, alpha-2-macroglobulin)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_matrix.to_csv('dependency_matrix.txt', sep='\t', mode='a')